# Pré-traîtement des images de la base BraTS 2019 pour l'entraînement de réseaux de neurones dédiés à la segmentation de tumeurs

Le code présenté ci-dessous permet le redimensionnement et la normalisation des images issues de la base de données BraTS 2019.

## Introduction

### Configuration

Ce code a été conçu et testé avec la configuration suivante : 
-  Python 3.10
-  Pandas 1.5
-  NiBabel 5.0
-  NiLearn 0.10

Le téléchargement et la décompression des données sont assurés par les fonctions **wget** et **unzip** qui peuvent être installées dans Windows à l'aide du logiciel ***Cygwin***

In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.image import resample_img
import time

### Téléchargement des données

Les données de la base sont récupérées sur les serveurs AWS de l'école Blent AI, puis décompressées dans le dossier d'où est lancé ce programme. L'archive compressée est ensuite supprimée.

In [ ]:
# Téléchargement des données
!wget -q https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/60fb61/brats_2019.zip
!unzip -qq brats_2019.zip -d ./
!rm brats_2019.zip

### Description des données originales

Les données de la base BraTS 2019 sont organisées de la façon suivante :  Pour chaque cas (335 cas au total), la base contient 5 images au format .nii, qui sont : 
-  L'image IRM avec pondération T1 (**'t1'**)
-  L'image IRM avec pondération T1 et augmentation du contraste (**'t1ce'**)
-  L'image IRM avec pondération T2 (**'t2'**)
-  L'image IRM avec pondération T2 et atténuation des fluides par inversion-récupération (**'flair'**)
-  La carte de segmentation (**'seg'**)

Ces données sont présentées sous la forme de volumes 3D de dimensions (240, 240, 155) voxels.

Les images IRM comportent toutes les acquisitions du cerveau, qui a été isolé (le crâne et les tissus autres que ceux du cerveau ont été retirés). Les intensités sont représentées au format *int16*, et valent 0 patrout en dehors du cerveau. Ces données ne sont pas normalisées, ainsi d'un cas à l'autre la plage de dynamique de l'intensité peut être très variable.

La carte de segmentation est aussi au format *int16*, et vaut 0 partout en dehors des tissus tumoraux. Ces tissus ont été segmentés à la main, en 3 catégories. Le label associé à une catégorie correspond à la valeur de la carte de segmentation en un point appartenant à un tissu de cette catégorie. Ces catégories sont : 
-  *Necrotic and non-enhancing tumor core* - (Label 1)
-  *Peritumoral edema* - (Label 2)
-  *GD-enhancing tumor* - (Label 4)

Se reporter à la [page de description du dataset BraTS 2019](https://www.med.upenn.edu/cbica/brats2019/data.html) pour plus d'informations.

Pour chaque cas, toutes ces images ont été recalées dans un repère commun, mais l'isolation des tissus du cerveau ayant été faite de façon indépendante pour chaque image, leurs contours sont parfois différents. De plus, la plupart des images n'atteignent pas le bord du volume de données.

### Données attendues

Afin d'entraîner les réseaux de neurones dans les meilleurs conditions, les données de la base doivent être normalisées et redimensionnées pour correspondre aux dimensions d'entrée de ces réseaux. Ces données doivent avoir une plage dynamique commune, un contraste proche, et occuper le plus grand volume possible au sein de l'image 3D, sans subir de déformation géométrique.

Les données d'un cas sont agrégées en un volume 3D à 4 canaux (un par type d'images IRM), dont la dimension est choisie par lutilisateur, et dans laquelle l'image du cerveau est centrée et est tangente au bord du volume de données dans au moins une des dimensions de l'espace. Les valeurs d'intensité de chaque image sont comprises entre -1 et 1, et la médiane des voxels non nuls vaut 0. Les données dont l'intensité est supérieure à la médiane sont écrasées pour correspondre à ces statistiques.

La carte de segmentation est présentée sous la forme d'un volume 3D, dans le même repère que les données, dans lequel chaque voxel est labelisé avec les valeurs suivantes : 
-  0 : Extérieur du cerveau
-  1 : Tissu sain
-  2 : *Necrotic and non-enhancing tumor core*
-  3 : *Peritumoral edema*
-  4 : *GD-enhancing tumor*

L'ensemble des informations sur chaque cas (données et segmentation) sont sauvegardées en un unique fichier .npz.'

Ci-dessous, l'utilisateur définit la dimension des images qu'il souhaite obtenir.

In [ ]:
target_shape = (240, 240, 144)

### Extraction des données de la base

La base de données BraTS 2019 est fournie avec un fichier .csv qui répertorie l'ensemble des cas contenus dans la base. Ce fichier se nomme **'./MICCAI_BraTS_2019_Data_Training/name_mapping.csv'**, et contient entre autre le champ **'Grade'**, qui indique le type de pathologie, et le champ **'BraTS_2019_subject_ID'** qui est l'identifiant du cas. Un second fichier .csv contient les données de survie des patients.

Chaque ensemble d'image relatif à un cas est stocké dans le dossier **./MICCAI_BraTS_2019_Data_Training/{Grade}/{BraTS_2019_subject_ID}**, où **{Grade}** et **{BraTS_2019_subject_ID}** sont les valeurs extraites du fichier .csv

Dans ces dossiers, les images sont nommées **{BraTS_2019_subject_ID}_{Channel}.nii** où **{Channel}** indique le type d'image (peut être pris parmi la liste ['flair', 't1', 't1ce', 't2', 'seg']).

Afin de simplifier la navigation dans la base pour la suite, on construit la liste **path_to_data** qui contient pour chaque cas le chemin vers le dossier contenant les données, ainsi que l'identifiant du cas, utile pour reconstruire le nom des fichiers.

Puisque à la fin du pré-traîtement la base de données originale sera supprimée pour réduire le volume de données stockées, on extrait les fichiers .csv du dossier **./MICCAI_BraTS_2019_Data_Training/**.

In [ ]:
# Déplacement des fichiers .csv
!mv ./MICCAI_BraTS_2019_Data_Training/*.csv ./

# Récupération des chemins des données
name_mapping = pd.read_csv("name_mapping.csv", header=0)
path_to_data = []
for i in range(name_mapping.shape[0]):
    path_to_data.append([os.path.join('MICCAI_BraTS_2019_Data_Training', name_mapping["Grade"][i], name_mapping["BraTS_2019_subject_ID"][i]),name_mapping["BraTS_2019_subject_ID"][i]])                      

## Transformation des images

Ici on discute la façon dont les images sont normalisées et déplacées dans une enveloppe commune aux dimensions déterminées précédemment.

### Enveloppe parallélépipédique d'une image

Les données de la base originale ne remplissent pas tout l'espace qu'elles pourraient, en n'atteignant pas les bords des volumes de données. Après le pré-traîtement, on souhaite que toutes les images aient cette propriété. Pour cela on va rechercher pour chaque cas la plus petite enveloppe parallélépipédique rectangle (suivant les axes de l'image) englobant toutes les images du cas, puis calculer la transformation qui amène cette enveloppe aux bords du volume de données dont on a choisi les dimensions, tout en gardant cette enveloppe centrée, entièrement incluse dans le volume de données, et sans en modifier les proportions.

Le calcul de l'enveloppe d'une image est simple : suivant chaque axe, on détermine la plus petite et la plus grande coordonnée pour laquelle l'intensité de l'image est non nulle. Les 6 coordonnées ainsi obtenuent définissent les 6 faces de l'enveloppe parallélépipédique d'une image. Par la suite on calcule la plus petite enveloppe commune à toutes les images d'un cas, en récupérant les valeurs extrèmes obtenues sur chacune des images de ce cas.

In [ ]:
def find_bounding_box(img):
    # Recherche l'enveloppe parallélépipédique de l'image
    
    b_box = np.zeros((3,2))
    NZ = np.nonzero(img)
    for k in range(3):
        b_box[k,:] = np.asarray([np.min(NZ[k]), np.max(NZ[k])])

    return b_box

### Calcul de la transformation

Les fichiers .nii (le format original des images de la base BraTS 2019) contiennent, en plus des informations des voxels et des métadonnées, une matrice nommée *affine* permettant de spécifier la position de l'image dans un système de coordonnées. Les caractéristiques de cette matrice *affine* et ses relations avec le système de coordonnées sont expliquées sur [cette page](https://nipy.org/nibabel/coordinate_systems.html). La librairie NiLearn permet de ré-échantillonner une image dans un autre système de coordonnées. 

On suppose que la matrice affine de l'image que l'on souhaite obtenir est la matrice identité (matrice carrée de taille 4).

Pour déterminer la matrice *affine* de l'image originale dans le repère de l'image souhaitée, on doit calculer la transformation qui amène cette image originale sur l'image souhaitée. On peut décomposer cette transformation en 3 transformations élémentaires : 
-  Tout d'abord, on translate l'image originale de façon à ce que le centre de l'enveloppe parallélépipédique se retrouve au centre du repère
-  On effectue une homotétie par un facteur d'échelle qui permet de grandir ou rétrécir les données pour qu'elles prennent le plus grand volume possible dans le bloc de données final, mais sans que l'enveloppe n'en dépasse.
-  On translate de nouveau l'image obtenue pour amener le centre de l'enveloppe transformée au centre du volume de données.

Chacune de ces transformations se traduit par une matrice carrée de taille 4, et la transformation globale n'est que le produit matriciel de toutes ces transformations.

La fonction ci dessous appelle la fonction de recherche de la plus petite enveloppe commune, puis calcule les 3 matrices des transformations élémentaires, avant de construire la matrice *affine* de la transformation globale.

In [ ]:
def compute_affine_transform(best_b_box, target_shape):
    # Calcule la transformation affine 4x4 pour amener les données de leur espace initial vers la taille d'image décidée, sans déformation

    scale_vec = target_shape/(best_b_box[:,1]-best_b_box[:,0])
    scale_factor = min(scale_vec)
    center_target = np.transpose(target_shape/2)
    center_img = np.transpose((best_b_box[:,0] + best_b_box[:,1])/2)
    
    center_affine = np.eye(4)
    center_affine[:3,-1] = -center_img
    
    scale_affine = np.eye(4)*scale_factor
    scale_affine[3,3] = 1
    
    translate_affine = np.eye(4)
    translate_affine[:3,-1] = center_target
    
    target_affine = np.dot(translate_affine, np.dot(scale_affine, center_affine))
    
    return target_affine

### Normalisation des intensités

Les images présentes dans la base BraTS 2019 ont toutes des plages d'intensité très différentes les unes des autres. Afin que cela influe le moins possible sur la qualité de l'entraînement des réseaux de neurones, il est nécessaire de normaliser ces intensités.

Dans la plupart des images, le tissu tumoral est celui présentant la plus grande intensité, qui parfois est très supérieure à celle des tissus sains. Certains voxels atteignent parfois la valeur de saturation de l'instrument (32767, valeur maximale d'un *int16*). Cependant le volume de tissu tumoral est au moins *presque toujours* inférieur au volume des tissus sains. Puisque tous les voxels situés en dehors du cerveau ont une intensité de 0, l'intensité médiane des voxels non-nuls est une mesure robuste du niveau global de l'image. 

On souhaite que toutes les intensités soient comprises entre -1 et 1. On va donc réduire les intensités en les divisant par la valeur de la médiane des voxels non nuls, et soustraire 1. Ainsi l'extérieur du cerveau a une intensité de -1 et tous les voxels dont l'intensité est inférieure à la médiane obtiennent une intensité comprise entre -1 et 0. Pour tous les voxels au dessus de cette médiane, on applique un écrasement de la dynamique pour s'assurer que la valeur de l'intensité reste inférieure à 1. La transformation est la suivante : avec $x$ l'intensité  réduite du voxel, on applique : 
\begin{equation}
x \rightarrow 
\begin{cases}
x & \text{ si } x \leq 0 &\\
1-e^{-x} & \text{ si } x \gt 0
\end{cases}
\end{equation}

Avec cette transformation, on s'assure que l'ordre des intensités est respecté, qu'elles sont comprises entre -1 et 1, et puisque cette transformation est $C^1$, il n'y a pas de rupture de pente dans la courbe des intensités.

In [ ]:
def normalize(img):
    img = img/(np.median(img[np.nonzero(img)])) - 1
    img = np.where(img > 0, 1-np.exp(-img), img)
    return img

## Application du pré-traîtement

Dans cette section, on détaille l'ordre dans lequel sont effectuée les opération de pré-traîtement

### Fonction de pré-traîtement d'un cas

La fonction ci-dessous s'occupe de réaliser les opérations de pré-traîtement d'un cas.

Dans un premier temps, on charge chacune des images relatives à ce cas, et on calcule leurs enveloppes individuelles. Puis on calcule la plus petite enveloppe commune à toutes ces images, avant de s'en servir pour déterminer la matrice *affine* de transformation.

Dans un second temps, on identifie les voxels appartenant à du tissus sain. Pour cela, on sélectionne tous les voxels qui sont à la fois non nuls dans au moins une des images IRM, et nuls dans la carte de segmentation.

Ensuite, pour chaque image IRM, on normalise les données, et on applique la transformation. Toutes ces images sont regroupées dans un unique tenseur nommé **data**.

Enfin, on re-labellise la carte de segmentation pour coller aux spécifications des données attendues, et on lui applique à elle aussi la transformation. Pour éviter que cette carte contienne des valeurs non-entières dues à l'interpolation lors de la transformation, on arrondit chaque voxel à l'entier le plus proche.

Enfin, on sauvegarde les données et la carte de segmentation dans un unique fichier nommé **./preprocessed_data/{BraTS_2019_subject_ID}.npz**, où **{BraTS_2019_subject_ID}** est le nom du cas issu du fichier .csv. 

Cette fonction prend en entrée le chemin du dossier du cas **path**, l'identifiant dans la base qui sert de nom de fichier **file_name** ainsi que la dimension souhaitée des images pré-traitées **target_shape**

In [ ]:
def preprocess(path, file_name, target_shape):
    target_shape = np.asarray(target_shape)
    data_tmp = np.zeros((240,240,155,5))
    channels = ["flair","t1", "t1ce", "t2", "seg"] 
    bounding_boxes = np.zeros((3,2,5))
    
    # Chargement des images du cas et calcul des enveloppes individuelles
    for k in range(5):
        loaded = nib.load(os.path.join(path, file_name + "_" + channels[k] + ".nii"))
        img = loaded.get_fdata()
        bounding_boxes[:,:,k] = find_bounding_box(img)
        data_tmp[:,:,:,k] = img
    
    # Calcul de la meilleur enveloppe parallélépipédique et de la transformation affine
    best_b_box = np.zeros((3,2))
    best_b_box[:,0] = bounding_boxes[:,0,:].min(axis=1)
    best_b_box[:,1] = bounding_boxes[:,1,:].max(axis=1)
    affine = compute_affine_transform(best_b_box, target_shape)
    
    # Calcul du masque des tissus sains et intégration dans la carte de segmentation
    data_tmp[:,:,:,4] = np.where((data_tmp.sum(axis=3) != 0) & (data_tmp[:,:,:,4] == 0), 5, data_tmp[:,:,:,4])
    
    # Normalisation et recalage des images de données
    data = np.zeros((target_shape[0], target_shape[1], target_shape[2], 4))
    for k in range(4):
        data_tmp[:,:,:,k] = normalize(data_tmp[:,:,:,k])
        img = nib.Nifti1Image(data_tmp[:,:,:,k], affine=affine)
        data[:,:,:,k] = resample_img(img, target_affine=np.eye(4), target_shape=target_shape, fill_value=-1).get_fdata()
    
    # Relabellisation et recalage de la nouvelle carte de segmentation
    seg_tmp = data_tmp[:,:,:,4]
    seg_tmp[seg_tmp == 2] = 3
    seg_tmp[seg_tmp == 1] = 2
    seg_tmp[seg_tmp == 5] = 1
    img = nib.Nifti1Image(seg_tmp, affine=affine)
    seg = resample_img(img, target_affine=np.eye(4), target_shape=target_shape, fill_value=0).get_fdata()
    seg = np.rint(seg)
    
    # Sauvegarde des données
    np.savez_compressed(os.path.join('preprocessed_data', file_name), data=data.astype("float32"), seg=seg.astype("byte"))
    
    return

### Exécution du pré-traîtement

Il ne nous reste plus qu'à boucler sur tous les cas de la base pour leur appliquer le pré-traîtement.

Dans un premier temps, s'il n'existe pas encore, le dossier **./preprocessed_data/** est créé.

Ensuite, pour chaque paire chemin/identifiant dans la liste **path_to_data**, on applique la fonction de pré-processing. Tous les 5 cas, on affiche les informations d'avancement, de temps écoulé, et de temps restant estimé.

Lorsque tous les cas ont été traîtés, on supprime les données de la base originale et on affiche la durée totale du pré-traîtement.

In [ ]:
os.makedirs('preprocessed_data', exist_ok=True)
start_time = time.time()

for i in range(len(path_to_data)):
    path = path_to_data[i][0]
    file_name = path_to_data[i][1]
    preprocess(path, file_name, target_shape)
    
    if (i+1)%5 == 0 :
        stop_time = time.time()
        avancement = (i+1)/(len(path_to_data))
        duree = stop_time - start_time
        duree_totale = duree / avancement
        ETA = duree_totale - duree
        print("Avancement : {0:.2f}%, Temps écoulé : {1:d}h {2:02d}m {3:02d}s, Temps restant estimé: {4:d}h {5:02d}m {6:02d}s"
              .format(round(100*avancement,2), int(duree)//3600, (int(duree)//60)%60, int(duree)%60, int(ETA)//3600, (int(ETA)//60)%60, int(ETA)%60))

# Suppression de la base originale
!rm -rf MICCAI_BraTS_2019_Data_Training

duree = time.time() - start_time
print("Pré-traîtement terminé. Durée totale : {0:d}h {1:02d}m {2:02d}s".format(int(duree)//3600, (int(duree)//60)%60, int(duree)%60))